<a href="https://colab.research.google.com/github/micahks/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Micah_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=False` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

# Import

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [ ]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [ ]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)
df['BLOCK'] = df['BLOCK'].astype(str)
df['LOT'] = df['LOT'].astype(str)
df['TAX_CLASS_AT_TIME_OF_SALE'] = df['TAX_CLASS_AT_TIME_OF_SALE'].astype(str)
df['ZIP_CODE'] = df['ZIP_CODE'].astype(str)

In [ ]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                              object
LOT                                object
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                           object
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE          object
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [ ]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

# Wrangling

In [ ]:
#SET INDEX
#change df to df_int so I can continue to change wrangle function with overwriting df constantly

df_int = df.set_index('SALE_DATE')

In [ ]:
def wrangle(df):

  # filtering for one family dwellings and price above 100K and below 2M
  df = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') &
          (df['SALE_PRICE'] > 100000) &
          (df['SALE_PRICE'] < 2000000)]

  # drop columns that have mostly null values        
  null_cols = [col for col in (df.columns) if (df[col].isna().sum() > 0)]
  df.drop(columns=null_cols, inplace=True) 

  # drop columns with high cardinality and any columns with only 1 value ('Building Calss Category')
  hc_cols = [col for col in df.select_dtypes('object').columns
             if (df[col].nunique() > 15) | (df[col].nunique() == 1) ]
  df.drop(columns=hc_cols, inplace=True) 



  return df

In [ ]:
df = wrangle(df_int)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
print(df.shape)
df

(3151, 11)


,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE_DATE,,,,,,,,,,,
01/01/2019,3,OTHER,1,A9,1.0,0.0,1.0,1325.0,1930.0,A9,550000
01/01/2019,4,OTHER,1,A1,1.0,0.0,1.0,2001.0,1940.0,A1,200000
01/02/2019,2,OTHER,1,A1,1.0,0.0,1.0,2043.0,1925.0,A1,810000
01/02/2019,3,OTHER,1,A1,1.0,0.0,1.0,2680.0,1899.0,A1,125000
01/02/2019,3,OTHER,1,A5,1.0,0.0,1.0,1872.0,1940.0,A5,620000
...,...,...,...,...,...,...,...,...,...,...,...
04/30/2019,4,OTHER,1,A2,1.0,0.0,1.0,1478.0,1925.0,A2,635000
04/30/2019,4,OTHER,1,A1,1.0,0.0,1.0,1280.0,1930.0,A1,514000
04/30/2019,4,OTHER,1,A0,1.0,0.0,1.0,1333.0,1945.0,A0,635000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3151 entries, 01/01/2019 to 04/30/2019
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3151 non-null   object 
 1   NEIGHBORHOOD                    3151 non-null   object 
 2   TAX_CLASS_AT_PRESENT            3151 non-null   object 
 3   BUILDING_CLASS_AT_PRESENT       3151 non-null   object 
 4   RESIDENTIAL_UNITS               3151 non-null   float64
 5   COMMERCIAL_UNITS                3151 non-null   float64
 6   TOTAL_UNITS                     3151 non-null   float64
 7   GROSS_SQUARE_FEET               3151 non-null   float64
 8   YEAR_BUILT                      3151 non-null   float64
 9   BUILDING_CLASS_AT_TIME_OF_SALE  3151 non-null   object 
 10  SALE_PRICE                      3151 non-null   int64  
dtypes: float64(5), int64(1), object(5)
memory usage: 295.4+ KB


#Split Data

##Split target and feature matrix

In [ ]:
target = 'SALE_PRICE'
y = df[target]
X = df.drop(columns=target)

## Split Training and Test Sets

In [ ]:
cutoff = '04/01/2019'
mask = X.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

# Baseline

In [ ]:
y_pred = [y_train.mean()] * len(y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error

print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

Baseline MAE: 214721.52773001452


# One Hot Encoding (OHE)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3151 entries, 01/01/2019 to 04/30/2019
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3151 non-null   object 
 1   NEIGHBORHOOD                    3151 non-null   object 
 2   TAX_CLASS_AT_PRESENT            3151 non-null   object 
 3   BUILDING_CLASS_AT_PRESENT       3151 non-null   object 
 4   RESIDENTIAL_UNITS               3151 non-null   float64
 5   COMMERCIAL_UNITS                3151 non-null   float64
 6   TOTAL_UNITS                     3151 non-null   float64
 7   GROSS_SQUARE_FEET               3151 non-null   float64
 8   YEAR_BUILT                      3151 non-null   float64
 9   BUILDING_CLASS_AT_TIME_OF_SALE  3151 non-null   object 
 10  SALE_PRICE                      3151 non-null   int64  
dtypes: float64(5), int64(1), object(5)
memory usage: 295.4+ KB


In [ ]:
### OHE on Training Data

# Step 1: Import the transformer class
from category_encoders import OneHotEncoder, OrdinalEncoder

# Step 2: Instantiate the transformer
ohe = OneHotEncoder(use_cat_names=True)

# Step 3: Fit your TRAINING data to the transfomer
ohe.fit(X_train)

# Step 4: Transform
XT_train = ohe.transform(X_train)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
print(len(XT_train.columns))
XT_train

42


,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
SALE_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01/01/2019,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,1325.0,1930.0,1,0,0,0,0,0,0,0,0,0,0
01/01/2019,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,2001.0,1940.0,0,1,0,0,0,0,0,0,0,0,0
01/02/2019,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,2043.0,1925.0,0,1,0,0,0,0,0,0,0,0,0
01/02/2019,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,2680.0,1899.0,0,1,0,0,0,0,0,0,0,0,0
01/02/2019,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,1872.0,1940.0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
03/29/2019,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,1.0,921.0,1950.0,0,0,0,0,1,0,0,0,0,0,0
03/29/2019,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,2128.0,1980.0,0,0,1,0,0,0,0,0,0,0,0
03/29/2019,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,1807.0,2018.0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
##OHE on Test Data

XT_test = ohe.transform(X_test)

# SelectKBest

## How many Features should I select?

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

maes = []
for k in range(1, len(XT_train.columns)+1):
    
    selector = SelectKBest(score_func=f_regression, k=k)
    XT_train_selected = selector.fit_transform(XT_train, y_train)
    XT_test_selected = selector.transform(XT_test)

    model = LinearRegression()
    model.fit(XT_train_selected, y_train)
    y_pred = model.predict(XT_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    maes.append(mae)

maes

[183640.5858012459,
 182569.26269991108,
 182569.26269991108,
 173706.2603593521,
 174555.8256347396,
 173087.31802707532,
 172936.73410780515,
 172278.7866149667,
 172745.2270457845,
 172738.4564227243,
 173799.145817194,
 174072.3682229649,
 174065.3478433882,
 173813.32421522267,
 173813.32421521936,
 173651.80437469936,
 173651.8043747026,
 172822.68942579476,
 172818.72254944537,
 172818.7225494456,
 172818.72254944537,
 172818.72254944564,
 172647.48841871755,
 172571.8268260701,
 172529.18120112413,
 172501.81162093315,
 173687.3363424268,
 173824.6664001756,
 173824.66640017473,
 172715.7749574841,
 172692.41049239843,
 172208.1692713277,
 172208.16927132697,
 172815.43706467873,
 172289.4791897939,
 172356.15021709967,
 172345.5684964449,
 172422.2329488705,
 172379.5899381844,
 172375.20566956647,
 172375.205669566,
 172375.2056695649]

Looking at the print out we'll go with K=8

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(k=8)

XTT_train = selector.fit_transform(XT_train, y_train)
XTT_test = selector.transform(XT_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [ ]:
XTT_train.shape, XTT_test.shape

((2507, 8), (644, 8))

# Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=5, normalize=False)
model.fit(XTT_train, y_train)

Ridge(alpha=5, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [ ]:
print('RIDGE train MAE', mean_absolute_error(y_train, model.predict(XTT_train)))
print('RIDGE test MAE', mean_absolute_error(y_test, model.predict(XTT_test)))

RIDGE train MAE 207183.93241360458
RIDGE test MAE 206763.60965669656
